# Load data

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#### here you can find User-Agent not to be blocked
https://developers.whatismybrowser.com/useragents/explore/software_name/googlebot/

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

### 1. Go to the real estate website
- avito.ru
- ...

### 2. Filter the property by criteria (like zone, price filters, number of rooms, etc)

In [4]:
# https://www.avito.ru/moskva_i_mo/kvartiry/prodam/novostroyka-ASgBAQICAUSSA8YQAUDmBxSOUg?q=%D0%B6%D0%BA+%D0%B8%D0%BD%D0%BD%D0%BE%D0%B2%D0%B0%D1%86%D0%B8%D1%8F&f=ASgBAQICAUSSA8YQAkDmBxSOUsoIhJasAZSsAYhZhlmEWYJZgFn~WA&p=1

In [5]:
avito = "https://www.avito.ru/moskva_i_mo/kvartiry/prodam/novostroyka-ASgBAQICAUSSA8YQAUDmBxSOUg?q=%D0%B6%D0%BA+%D0%B8%D0%BD%D0%BD%D0%BE%D0%B2%D0%B0%D1%86%D0%B8%D1%8F&f=ASgBAQICAUSSA8YQA0DmBxSOUsoIhJasAZSsAYhZhlmEWYJZgFn~WJC~DRSWrjU"
response = get(avito, headers=headers)

In [6]:
response

<Response [200]>

### 3. Get HTML Page Code

In [7]:
print(response.text[:1000])


<!DOCTYPE html>
   
<html> <head> <script>
 try {
 if ('PerformanceLongTaskTiming' in window) {
 var globalStats = window.__statsLongTasks = { tasks: [] };
 globalStats.observer = new PerformanceObserver(function(list) {
 globalStats.tasks = globalStats.tasks.concat(list.getEntries());
 });
 globalStats.observer.observe({ entryTypes: ['longtask'] });
 }
 if (PerformanceObserver && (PerformanceObserver.supportedEntryTypes || []).some(function(e) {
 return e === 'element'
 })) {
 if (!window.oet) {
 window.oet = [];
 }
 new PerformanceObserver(function(l) {
 window.oet.push.apply(window.oet, l.getEntries());
 }).observe({ entryTypes: ['element'] });
 }
 } catch (e) {
 console.error(e);
 }
 </script>
  <script>
 window.dataLayer = [{"userAuth":0,"pageType":"Other","cityName":"moskva_i_mo","regionName":"R11"}];
 (function(w, d, s, l, i) {
 w[l] = w[l] || [];
 w[l].push({
 'gtm.start': new Date().getTime(),
 event: 'gtm.js'
 });
 var f = d.getElementsByTagName(s)[0],
 j = d.createElement(s

In [8]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
html_soup


<!DOCTYPE html>

<html> <head> <script>
 try {
 if ('PerformanceLongTaskTiming' in window) {
 var globalStats = window.__statsLongTasks = { tasks: [] };
 globalStats.observer = new PerformanceObserver(function(list) {
 globalStats.tasks = globalStats.tasks.concat(list.getEntries());
 });
 globalStats.observer.observe({ entryTypes: ['longtask'] });
 }
 if (PerformanceObserver && (PerformanceObserver.supportedEntryTypes || []).some(function(e) {
 return e === 'element'
 })) {
 if (!window.oet) {
 window.oet = [];
 }
 new PerformanceObserver(function(l) {
 window.oet.push.apply(window.oet, l.getEntries());
 }).observe({ entryTypes: ['element'] });
 }
 } catch (e) {
 console.error(e);
 }
 </script>
<script>
 window.dataLayer = [{"userAuth":0,"pageType":"Other","cityName":"moskva_i_mo","regionName":"R11"}];
 (function(w, d, s, l, i) {
 w[l] = w[l] || [];
 w[l].push({
 'gtm.start': new Date().getTime(),
 event: 'gtm.js'
 });
 var f = d.getElementsByTagName(s)[0],
 j = d.createElement(s),
 d

### 4. Find the "div class" of the property object that contains all the description 

In [11]:
house_containers = html_soup.find_all('div', class_="description item_table-description")
house_containers

[<div class="description item_table-description">
 <div class="snippet-title-row"><h3 class="snippet-title" data-marker="item-title" data-shape="default"><a class="snippet-link" href="/novoivanovskoe/kvartiry/studiya_22.3_m_1025_et._1910982421" itemprop="url" target="_blank" title="Студия, 22.3 м², 10/25 эт. в Новоивановском">Студия, 22.3 м², 10/25 эт.</a></h3></div> <div class="snippet-price-row"><span class="snippet-price" data-marker="item-price" data-shape="default">
  3 800 000  ₽
   </span></div>
 <div class="snippet-badges">
 </div>
 <div class="address" data-marker="item-address">
 <div><div data-marker="item-development-name" data-shape="default">
   ЖК «Инновация»  
  </div></div> <div><div class="item-address" data-shape="default"><div itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span class="item-address__string">
  ул. Эйнштейна, стр. 3
  </span><div class="item-address-georeferences"><span class="item-address-georeferences"><span class="item-

#### find attribute

In [12]:
first = house_containers[0]
first.find_all('span')

[<span class="snippet-price" data-marker="item-price" data-shape="default">
  3 800 000  ₽
   </span>,
 <span class="item-address__string">
  ул. Эйнштейна, стр. 3
  </span>,
 <span class="item-address-georeferences"><span class="item-address-georeferences-item"><span class="item-address-georeferences-item__content">Новоивановское</span></span></span>,
 <span class="item-address-georeferences-item"><span class="item-address-georeferences-item__content">Новоивановское</span></span>,
 <span class="item-address-georeferences-item__content">Новоивановское</span>]

In [23]:
var_1 = first.find_all('span')[0].text
var_1

'\n 3 800 000  ₽\n  '

In [24]:
# clean attribute
var_1 = var_1.replace('\n','')
var_1

' 3 800 000  ₽  '

In [25]:
var_1 = var_1.replace(' ','')
var_1

'3800000₽'

In [26]:
var_1 = var_1.replace('₽','')
var_1

'3800000'

In [27]:
var_1 = int(''.join(itertools.takewhile(str.isdigit, var_1)))
var_1

3800000

In [28]:
print(var_1, type(var_1))

3800000 <class 'int'>


In [29]:
# location
location = first.find_all('div', class_="address")[0].text
location

'\n\n  ЖК «Инновация»  \n  \n ул. Эйнштейна, стр. 3\n Новоивановское\n'

In [30]:
location = location.replace('\n','')
location

'  ЖК «Инновация»     ул. Эйнштейна, стр. 3 Новоивановское'

In [31]:
location[21:100]

'ул. Эйнштейна, стр. 3 Новоивановское'

In [32]:
type_of_flat = first.find_all('div', class_="snippet-title-row")[0].text
type_of_flat

'Студия, 22.3 м², 10/25 эт.'

In [33]:
type_of_flat[0:type_of_flat.find(',')]

'Студия'

In [34]:
#getting all the links
for url in first.find_all('a'):
    print('https://www.avito.ru/' + url.get('href'))

https://www.avito.ru//novoivanovskoe/kvartiry/studiya_22.3_m_1025_et._1910982421


In [35]:
price = []
location = []
type_of_flat = []

### 5. Go through all pages by loop

In [36]:
%%time
from time import sleep
from random import randint

location2 = list()
price2 = list()
type_of_flat2 = list()

n_pages = 0

for page in range(0,1):
    n_pages += 1
    sapo_url = 'https://www.avito.ru/moskva_i_mo/kvartiry/prodam/novostroyka-ASgBAQICAUSSA8YQAUDmBxSOUg?q=%D0%B6%D0%BA+%D0%B8%D0%BD%D0%BD%D0%BE%D0%B2%D0%B0%D1%86%D0%B8%D1%8F&f=ASgBAQICAUSSA8YQA0DmBxSOUsoIhJasAZSsAYhZhlmEWYJZgFn~WJC~DRSWrjU'+'&p='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="description item_table-description")
    if house_containers != []:
        for container in house_containers:
            
    
            #location
            location = container.find_all('div', class_="address")[0].text
            location = location[21:100]
            location = location.replace('\n','')
            location = location.replace(' ','')
            location = location.replace('1','1,')
            location = location.replace('2','2,')
            location = location.replace('3','3,')
            location2.append(location)
            print(location)
            
            #price
            price = container.find_all('span', class_="snippet-price")[0].text
            price = price.replace('\n','')
            price = price.replace('₽  ','')
            price= price.replace(' ','')
            price2.append(price)
            print(price)
            
            #type_of_flat
            type_of_flat = container.find_all('div', class_="snippet-title-row")[0].text
            type_of_flat2.append(type_of_flat)
            print(type_of_flat)

ул.Эйнштейна,стр.3,Новоивановское
3800000
Студия, 22.3 м², 10/25 эт.
ул.Эйнштейна,стр.1,Новоивановское
3200000
Студия, 20 м², 13/22 эт.
ул.Эйнштейна,стр.2,Новоивановское
3850000
Студия, 24.1 м², 13/22 эт.
ул.Эйнштейна,стр.2,Новоивановское
4350000
Студия, 27.2 м², 13/14 эт.
ул.Эйнштейна,стр.2,Новоивановское
4700000
1-к квартира, 38 м², 14/24 эт.
ул.Эйнштейна,стр.1,Новоивановское
4600000
1-к квартира, 31.8 м², 16/22 эт.
ул.Эйнштейна,стр.1,Новоивановское
6399000
2-к квартира, 54 м², 3/22 эт.
ул.Эйнштейна,стр.1,Новоивановское
3200000
Студия, 19.2 м², 14/24 эт.
ул.Эйнштейна,стр.1,Новоивановское
2500000
Студия, 30 м², 3/3 эт.
ул.Эйнштейна,стр.1,Новоивановское
4600000
1-к квартира, 32 м², 12/22 эт.
ул.Эйнштейна,стр.1,Новоивановское
4800000
1-к квартира, 34.2 м², 22/22 эт.
ул.Эйнштейна,стр.1,Новоивановское
6900000
2-к квартира, 57.1 м², 12/15 эт.
овациярабочийпос.Новоивановское
4100000
Студия, 23 м², 16/25 эт.
ул.Эйнштейна,стр.2,Москва
5300000
1-к квартира, 40 м², 6/22 эт.
ул.Эйнштейна,стр.2,М

In [37]:
type_of_flat2

['Студия, 22.3 м², 10/25 эт.',
 'Студия, 20 м², 13/22 эт.',
 'Студия, 24.1 м², 13/22 эт.',
 'Студия, 27.2 м², 13/14 эт.',
 '1-к квартира, 38 м², 14/24 эт.',
 '1-к квартира, 31.8 м², 16/22 эт.',
 '2-к квартира, 54 м², 3/22 эт.',
 'Студия, 19.2 м², 14/24 эт.',
 'Студия, 30 м², 3/3 эт.',
 '1-к квартира, 32 м², 12/22 эт.',
 '1-к квартира, 34.2 м², 22/22 эт.',
 '2-к квартира, 57.1 м², 12/15 эт.',
 'Студия, 23 м², 16/25 эт.',
 '1-к квартира, 40 м², 6/22 эт.',
 'Студия, 25 м², 15/24 эт.',
 'Студия, 23 м², 15/16 эт.',
 'Студия, 22.3 м², 9/24 эт.',
 'Студия, 22 м², 15/22 эт.',
 'Студия, 19.5 м², 5/14 эт.',
 'Студия, 24 м², 3/25 эт.',
 'Студия, 22.8 м², 7/24 эт.',
 'Студия, 25.5 м², 2/14 эт.',
 'Студия, 23.1 м², 7/22 эт.',
 '1-к квартира, 35 м², 3/22 эт.',
 '1-к квартира, 31.5 м², 3/25 эт.',
 '1-к квартира, 34.6 м², 11/15 эт.',
 '1-к квартира, 42 м², 14/22 эт.',
 '2-к квартира, 57 м², 11/15 эт.',
 '2-к квартира, 55 м², 8/15 эт.',
 '2-к квартира, 54.7 м², 21/22 эт.',
 'Студия, 24.1 м², 6/22 эт.',

#### Put data to dataframe

In [38]:
import pandas as pd

cols = ['location', 'price', 'type_of_flat']

ZKI_Innovation = pd.DataFrame({'location': location2,
                           'price': price2,
                           'type_of_flat': type_of_flat2})[cols]
ZKI_Innovation

,location,price,type_of_flat
0,"ул.Эйнштейна,стр.3,Новоивановское",3800000,"Студия, 22.3 м², 10/25 эт."
1,"ул.Эйнштейна,стр.1,Новоивановское",3200000,"Студия, 20 м², 13/22 эт."
2,"ул.Эйнштейна,стр.2,Новоивановское",3850000,"Студия, 24.1 м², 13/22 эт."
3,"ул.Эйнштейна,стр.2,Новоивановское",4350000,"Студия, 27.2 м², 13/14 эт."
4,"ул.Эйнштейна,стр.2,Новоивановское",4700000,"1-к квартира, 38 м², 14/24 эт."
5,"ул.Эйнштейна,стр.1,Новоивановское",4600000,"1-к квартира, 31.8 м², 16/22 эт."
6,"ул.Эйнштейна,стр.1,Новоивановское",6399000,"2-к квартира, 54 м², 3/22 эт."
7,"ул.Эйнштейна,стр.1,Новоивановское",3200000,"Студия, 19.2 м², 14/24 эт."
8,"ул.Эйнштейна,стр.1,Новоивановское",2500000,"Студия, 30 м², 3/3 эт."
9,"ул.Эйнштейна,стр.1,Новоивановское",4600000,"1-к квартира, 32 м², 12/22 эт."


In [41]:
# save data
ZKI_Innovation.to_csv("dirty_data.csv", index=False)

In [40]:
ZKI_Innovation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 3 columns):
location        34 non-null object
price           34 non-null object
type_of_flat    34 non-null object
dtypes: object(3)
memory usage: 896.0+ bytes
